# Install

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 6.0 MB/s eta 0:00:00


# Imports

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
model_path = "ibm-granite/granite-embedding-125m-english"
df = pd.read_parquet("hf://datasets/bghira/photo-concept-bucket/photo-concept-bucket.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = SentenceTransformer(model_path)

modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/466k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/249M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
input_queries = [
    ' Who made the song My achy breaky heart? ',
    'summit define'
    ]

input_passages = [
    "Achy Breaky Heart is a country song written by Don Von Tress. Originally titled Don't Tell My Heart and performed by The Marcy Brothers in 1991. ",
    "Definition of summit for English Language Learners. : 1 the highest point of a mountain : the top of a mountain. : 2 the highest level. : 3 a meeting or series of meetings between the leaders of two or more governments."
    ]


In [ ]:
query_embeddings = model.encode(input_queries)
passage_embeddings = model.encode(input_passages)

In [ ]:
query_embeddings

array([[-0.00498647, -0.00575449,  0.00832645, ...,  0.02279777,
         0.04410243, -0.03797143],
       [-0.02497113, -0.01758112, -0.03495851, ...,  0.00884372,
        -0.02343597, -0.01375866]], dtype=float32)

In [ ]:
passage_embeddings

array([[-0.01735796, -0.00873789,  0.00414625, ...,  0.01451728,
         0.0176454 , -0.02187591],
       [-0.02364596, -0.00885225, -0.03927733, ...,  0.015592  ,
         0.02261679, -0.0111675 ]], dtype=float32)

In [ ]:
print(util.cos_sim(query_embeddings, passage_embeddings))

tensor([[0.9319, 0.6355],
        [0.6424, 0.8791]])


In [ ]:
len(query_embeddings[0])

768

In [ ]:
df2 = df[['alt', 'colors', 'main_color', 'cogvlm_caption', 'url']]
df2.head()

,alt,colors,main_color,cogvlm_caption,url
271389,Person Fixing Car,"[#3a3a3a, #8c8c8c, #cacaca]","[89, 89, 89]",A person is working underneath a car in a gara...,https://images.pexels.com/photos/474/black-and...
131264,Two Men Doing Ufc,"[#060606, #34271d, #4a4c47, #693b15, #837f72]","[64, 48, 33]",Two fighters are engaged in a grappling match ...,https://images.pexels.com/photos/650/people-me...
1172051,Gray Fish Illustration,"[#05061f, #0d1c81, #222fbc, #9aa2cf, #dbe4f9]","[22, 21, 65]",A serene underwater scene showcases a fish swi...,https://images.pexels.com/photos/656/sea-water...
381714,Brown Diving Suit and Helmet,"[#06070a, #3f362b, #71624d, #9d8059, #bfa27a]","[92, 76, 58]","An old-fashioned diving helmet, with a circula...",https://images.pexels.com/photos/718/vintage-h...
706881,Two Women Sitting on Brown Wooden Park Bench,"[#12140e, #37432e, #505c3c, #758538, #888570]","[42, 42, 27]",Two individuals sit on a wooden bench in a par...,https://images.pexels.com/photos/798/bench-peo...


# Pinecone

In [ ]:
pinecone_url = "XXXXX"

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="XXXXX")
index = pc.Index("ibm-granite")

In [ ]:
query_embeddings = model.encode(list(df2['cogvlm_caption'][:100]))

In [ ]:
index.upsert(
    vectors=query_embeddings,
    namespace= "ns1"
)